# Murmeln in der Schüssel (Das Prinzip von d'Alembert)

**Dies ist die Notebook-Version des YouTube-Videos**

Wir betrachten die eindimensionale Bewegung von Murmeln in einer Schüssel.

Die Schüssel sei durch einen Parameter in der $x, y$-Ebene darstellbar: $x=x(\theta), z=z(\theta)$. Hierbei ist $\theta = \theta(t)$.

<center><img src="figs/pendel_sketch.png" width=400 height=300></center>

Wir wollen die Bewegungsgleichungen für $\theta(t)$ und damit für $x(\theta)$ und $y(\theta)$ finden.

Mit $\vec{r} = \begin{pmatrix} x \\ z \end{pmatrix}$ lauten die Newtonschen Bewegungsgleichungen für das System:

$$
m \ddot{\vec{r}} = \vec{F}_g + \vec{F}_Z = \begin{pmatrix} 0 \\ -mg \end{pmatrix} + \vec{F}_Z \tag{1}
$$

Wir kennen allerdings $\vec{F}_Z$ nicht! Jedoch verrichten Zwangskräfte im vorliegenden Fall auf den *durch sie erlaubten Bahnen keine physikalische Arbeit* am System:

$$
\vec{r}_{\rm c} = \begin{pmatrix} x_{\rm c} \\ z_{\rm c} \end{pmatrix};\quad W_Z = \int \vec{F}_Z\,\text{d}\vec{r}_{\rm c} = \int \vec{F}_Z\vec{v}_{\rm c}\,\text{d}t \overset{!}{=}0
$$ (Anwendung des [Prinzips von d'Alembert](https://de.wikipedia.org/wiki/D%E2%80%99Alembertsches_Prinzip)). Dies hat $\vec{F}_Z\vec{v}_{\rm c}\equiv 0$ zur Folge.

Aus (1) können wir damit $\vec{F}_Z$ eliminieren:

$$
0 = m \ddot{\vec{r}} - \vec{F}_g - \vec{F}_Z \xRightarrow[]{\text{d'Alembert}} \left( m \ddot{\vec{r}_{\rm c}} - \vec{F}_g - \vec{F}_Z\right)\cdot{\vec{v}_{\rm c}} = \left( m \ddot{\vec{r}_{\rm c}} - \vec{F}_g\right)\cdot\vec{v}_{\rm c} = 0
$$

Aus 

$$
\left( m \ddot{\vec{r}_{\rm c}} - \vec{F}_g\right)\cdot{\vec{v}_{\rm c}} = 0 \tag{2}
$$

leiten wir die Bewegungsgleichung für $\theta(t)$ ab, *ohne* $\vec{F}_Z$ kennen zu müssen!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ma
import sympy as sp
import scipy.integrate as si

sp.init_printing()

## 1. Analytische Bewegungsgleichungen mit SymPy

### Notwendige SymPy Größen

In [ ]:
g, m, l, t = sp.symbols('g, m, l, t', real=True, positive=True)
theta = sp.symbols(r'\theta', cls=sp.Function)
theta = theta(t)
theta

In [ ]:
theta_d = sp.diff(theta, t)
theta_dd = sp.diff(theta, t, 2)
theta_dd

### Die Schüsselformen

In [ ]:
form = 'Zykloide'

if form == 'Kreis':
    xc = l * sp.sin(theta)
    zc = -l * sp.cos(theta)

if form == 'Parabel':
    xc = theta
    zc = l * (2 / sp.pi)**2 * theta**2 - l

if form == 'Zykloide':
    xc = l / 2 * (2 * theta + sp.sin(2 * theta))
    zc = l / 2 * (-1 - sp.cos(2 * theta))



### Implementation von $\left( m \ddot{\vec{r}_{\rm c}} - \vec{F}_g\right)\cdot{\vec{v}_{\rm c}} = 0$

In [ ]:
# Kurvenvektor
rc = sp.Matrix([xc, zc])
rc

In [ ]:
# Kurvengeschwindigkeit
vc = sp.diff(rc, t)
vc

In [ ]:
# Gewichtskraft
Fg = sp.Matrix([0, -m * g])
Fg

In [ ]:
# Skalarprodukt zwischen Newtongleichung und Geschwindigkeit
ngl = (m * sp.diff(rc, t, 2) - Fg).dot(vc)
ngl = ngl.simplify()
sp.Eq(ngl, 0)

In [ ]:
# Bewegungsgleichung für theta (zweite Ableitung von theta)
bgl = sp.solve(ngl, theta_dd)[0]
sp.Eq(theta_dd, bgl)

## 2. Numerische Lösung der Bewegungsgleichung mit SciPy

In [ ]:
# Festlegung der Konstanten
const = { m : 1, l : 1, g : 9.81 }

bgl = bgl.subs(const)
xc = xc.subs(const)
zc = zc.subs(const)


In [ ]:
# Numpy-Funktionen für Positionen und zweite Ableitung von theta
theta_dd_n = sp.lambdify((theta, theta_d), bgl)
xc_n = sp.lambdify(theta, xc)
zc_n = sp.lambdify(theta, zc)

In [ ]:
zc_n(np.linspace(0.0, 1.0, 10))

Wir haben:
$$
\ddot{\theta}(t) = \lambda(\theta, \dot{\theta}(t), t)
$$
Definiere $\omega(t)=\dot{\theta}(t)$ und wir erhalten das gekoppelte DGL-System 1. Ordnung:

\begin{eqnarray}
\dot{\theta}(t) & = & \omega(t) \\
\dot{\omega}(t) & = & \lambda(\theta, \omega(t), t)\\
\end{eqnarray}

Definiere $S=(\theta(t), \omega(t))$ und weiter mit der numerischen Lösung mit `odeint` wie in vorherigen Teilen der Serie.

Schreibe eine Python-Funktion zur Berechnung von

$$
\frac{dS}{dt} = \begin{bmatrix} \dot{\theta}(t) \\ \dot{\omega}(t)\end{bmatrix} = \begin{bmatrix} \omega(t) \\ \lambda(\theta, \omega(t), t)\end{bmatrix}
$$

In [ ]:
def dSdt(S, t):
    theta_n, omega_n = S

    return [ omega_n,
             theta_dd_n(theta_n, omega_n) 
           ]

Bestimme die Zeiten zu denen die DGL gelöst werden soll, lege die Anfangsbedingungen fest und löse die DGL mit `odeint`:

In [ ]:
t_n = np.linspace(0., 20., 1000)

S0_1 = (-np.pi/3, 0)
sol_1 = si.odeint(dSdt, y0=S0_1, t=t_n)
theta_n_1 = sol_1[:,0]
xc_n_1 = xc_n(theta_n_1)
zc_n_1 = zc_n(theta_n_1)

S0_2 = (-np.pi/4, 0)
sol_2 = si.odeint(dSdt, y0=S0_2, t=t_n)
theta_n_2 = sol_2[:,0]
xc_n_2 = xc_n(theta_n_2)
zc_n_2 = zc_n(theta_n_2)

## 3. Visualisierung der Ergebnisse

### Einfache Plots

In [ ]:
plt.plot(t_n, theta_n_1, t_n, theta_n_2)

### Animation der Murmelbewegungen

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(xc_n(theta_n_1), zc_n(theta_n_1), color='black')
ax.set_xlim(-np.pi / 2, np.pi / 2)
ax.set_ylim(-1, 0)

m1, = plt.plot([], [], 'ro')
m2, = plt.plot([], [], 'bo')

writer = ma.PillowWriter(fps = 1000 / 20)

with writer.saving(fig, 'murmel.gif', dpi=100):
    for frame in range(1000):
        m1.set_data([xc_n(theta_n_1[frame])], [zc_n(theta_n_1[frame])])
        m2.set_data([xc_n(theta_n_2[frame])], [zc_n(theta_n_2[frame])])

        writer.grab_frame()